In [ ]:
##Ensemble Neural Network

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


csv_path = ("filepath")
df = pd.read_csv(csv_path)


FEATURES = [
    'Nb', 'Ca', 'Tm', 'S', 'Sb', 'U', 'Mo', 'Sm', 'I', 'Ag', 'Ir', 'Pu', 'Hf', 'Sc', 'Br', 'Ni',
    'Co', 'Zr', 'W', 'Tb', 'Mn', 'Fe', 'Cr', 'Pb', 'Si', 'Ba', 'Nd', 'Cs', 'N', 'Lu', 'Se', 'Pt',
    'Pr', 'Au', 'La', 'Re', 'Zn', 'Os', 'Ti', 'Be', 'Ce', 'Cu', 'In', 'Rb', 'Tl', 'F', 'Dy', 'V',
    'Er', 'Ho', 'Pd', 'H', 'B', 'C', 'Cl', 'P', 'Hg', 'Sr', 'Gd', 'Ga', 'K', 'Te', 'Th', 'Yb',
    'Al', 'Tc', 'Np', 'Ta', 'Rh', 'Ru', 'As', 'Cd', 'Li', 'Sn', 'Ge', 'Mg', 'Eu', 'Na', 'O', 'Y', 'Bi',
    'Avg_Atomic_Number','Average_Weight','Average_Electronegativity',
    'Magnetic_proportion','Entropy','average_period','avg_magnetic_moment',
    'average_group','Rare_Earth_proportion'
]
TARGET = 'Mean_TN_K'

X = df[FEATURES]
y = df[TARGET]


bins = pd.cut(
    y,
    bins=[-np.inf, 8, 20, 50, 125, 300, np.inf],
    labels=False
)


X_train, X_test, y_train, y_test, bins_train, bins_test = train_test_split(
    X, y, bins,
    test_size=0.2,
    random_state=42,
    stratify=bins
)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


train_bin_counts = pd.Series(bins_train).value_counts().sort_index()
minority_bin_size = train_bin_counts.min()
print("Train bin counts:\n", train_bin_counts)
print(f"Smallest bin size = {minority_bin_size}\n")


def create_model(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model


np.random.seed(42)
tf.random.set_seed(42)
os.environ['PYTHONHASHSEED'] = '42'


from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

keras_reg = KerasRegressor(build_fn=lambda: create_model(X_train_scaled.shape[1]), verbose=0)
param_grid = {
    'batch_size': [32, 64,128],
    'epochs': [200, 500,800,1200]
}
grid = GridSearchCV(
    estimator=keras_reg,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)
grid_result = grid.fit(X_train_scaled, y_train)
best_params = grid_result.best_params_
print("Best hyperparameters:", best_params)


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
r2_scores, mae_scores = [], []

for fold, (tr_idx, val_idx) in enumerate(cv.split(X_train_scaled, bins_train), 1):
    Xt, Xv = X_train_scaled[tr_idx], X_train_scaled[val_idx]
    yt, yv = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    bins_tr = bins_train.iloc[tr_idx]

    fold_counts = pd.Series(bins_tr).value_counts()
    fold_min = fold_counts.min()

    M = 5 
    val_preds = np.zeros((len(Xv), M))

    for i in range(M):
        idxs = []
        for b in fold_counts.index:
            b_idxs = np.where(bins_tr == b)[0]
            if len(b_idxs) <= fold_min:
                sel = b_idxs
            else:
                sel = np.random.RandomState(100+i).choice(b_idxs, size=fold_min, replace=False)
            idxs.append(sel)
        idxs = np.concatenate(idxs)

        model = create_model(Xt.shape[1])
        model.fit(Xt[idxs], yt.iloc[idxs], epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=0)
        val_preds[:, i] = model.predict(Xv).flatten()

    yv_pred = val_preds.mean(axis=1)
    r2_scores.append(r2_score(yv, yv_pred))
    mae_scores.append(mean_absolute_error(yv, yv_pred))
    print(f" Fold {fold:>2}:   R²={r2_scores[-1]:.3f}, MAE={mae_scores[-1]:.1f}")

print("\nValidation performance (mean ± std):")
print(f" R²  : {np.mean(r2_scores):.3f} ± {np.std(r2_scores):.3f}")
print(f" MAE : {np.mean(mae_scores):.1f} ± {np.std(mae_scores):.1f}\n")


M_final = 30
models = []

for i in range(M_final):
    idxs = []
    for b in train_bin_counts.index:
        b_idxs = np.where(bins_train == b)[0]
        if len(b_idxs) <= minority_bin_size:
            sel = b_idxs
        else:
            sel = np.random.RandomState(200+i).choice(b_idxs, size=minority_bin_size, replace=False)
        idxs.append(sel)
    idxs = np.concatenate(idxs)

    model = create_model(X_train_scaled.shape[1])
    model.fit(X_train_scaled[idxs], y_train.iloc[idxs], epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=0)
    models.append(model)


all_preds = np.column_stack([m.predict(X_test_scaled).flatten() for m in models])
y_pred_mean = all_preds.mean(axis=1)
y_pred_std  = all_preds.std(axis=1)


print("Test performance:")
print(" R²  :", r2_score(y_test, y_pred_mean))
print(" MAE :", mean_absolute_error(y_test, y_pred_mean))
print(" RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_mean)))


sns.set(style="whitegrid")
plt.figure(figsize=(6,6))
plt.errorbar(
    y_test, y_pred_mean,
    yerr=y_pred_std,
    fmt='o',
    ecolor='lightgray',
    capsize=2,
    alpha=0.7
)
plt.plot(
    [y_test.min(), y_test.max()],
    [y_test.min(), y_test.max()],
    'r--', lw=1
)
plt.xlabel('Actual Mean Néel Temperature (K)')
plt.ylabel('Predicted Mean Néel Temperature (K)')
plt.title('Neural Network Ensemble Predictions ±1σ on TEST (Néel)')
plt.tight_layout()
plt.show()

In [ ]:
##Predicted vs Actual Neel temperature Plot

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import matplotlib as mpl
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

r2   = r2_score(y_test, y_pred_mean)
mae  = mean_absolute_error(y_test, y_pred_mean)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_mean))

mpl.rcParams.update({
    "axes.titlesize": 20,
    "axes.labelsize": 18,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
    "font.family": "sans-serif",
    "font.sans-serif": "Helvetica",
    "legend.fontsize": 13,
    "axes.linewidth": 1.2,
    "xtick.major.width": 1.1,
    "ytick.major.width": 1.1,
    "grid.alpha": 0.3,
    "grid.linestyle": "--"
})

norm = plt.Normalize(vmin=y_pred_std.min(), vmax=y_pred_std.max())
colors = cm.turbo(norm(y_pred_std))

fig, ax = plt.subplots(figsize=(7.1, 6))

scatter = ax.scatter(
    y_test, y_pred_mean,
    c=y_pred_std, cmap='turbo',
    edgecolor='black', linewidth=0.25,
    s=65, alpha=0.9
)

ax.errorbar(
    y_test, y_pred_mean,
    yerr=y_pred_std,
    fmt='none',
    ecolor='gray',
    alpha=0.2,
    capsize=2,
    linewidth=0.6,
    zorder=0
)

ax.plot(
    [y_test.min(), y_test.max()],
    [y_test.min(), y_test.max()],
    'r--', linewidth=1.5
)

x_min, x_max = y_test.min(), y_test.max()
ax.set_xlim([x_min - 40, x_max + 40])
ax.set_ylim([x_min - 40, x_max + 40])
ax.set_xlabel('Actual Néel Temperature (K)', labelpad=10)
ax.set_ylabel('Predicted Néel Temperature (K)', labelpad=10)
ax.set_title('ENN with Stratified Undersampling', pad=20)

stats_text = f"R² = {r2:.2f}\nMAE = {mae:.0f} K\nRMSE = {rmse:.0f} K"
ax.text(
    0.05, 0.95, stats_text,
    transform=ax.transAxes,
    fontsize=14,
    verticalalignment='top',
    bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5')
)

cbar = plt.colorbar(scatter, ax=ax, pad=0.03)
cbar.set_label('Prediction Std Deviation (K)', size=14)

plt.tight_layout(pad=2.5)
plt.show()

In [ ]:
## MAE distribution plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import expon

abs_errors = np.abs(y_test - y_pred_mean)
mae = np.mean(abs_errors)
mae_rounded = round(mae)

loc, scale = expon.fit(abs_errors)

cutoff = 300
filtered_errors = abs_errors[abs_errors <= cutoff]

plt.figure(figsize=(7, 5))
counts, bins, _ = plt.hist(
    filtered_errors, bins=30, density=True,
    alpha=0.85, color='royalblue', label='Histogram'
)

x_vals = np.linspace(0, cutoff, 500)
plt.plot(
    x_vals, expon.pdf(x_vals, loc, scale),
    'k-', lw=2, label='Fitted Exponential Distribution'
)

plt.axvline(
    mae, color='red', linestyle='--', lw=2,
    label=f'Mean Absolute Error: {mae_rounded} K'
)

plt.xlabel('Absolute Error (K)')
plt.ylabel('Density')
plt.title('Distribution of Absolute Errors', fontsize=15)
plt.legend(loc='upper right')
plt.grid(True)
plt.xlim(0, cutoff)
plt.tight_layout()
plt.show()

In [ ]:
## Feature Importance Plot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance

n_ensembles = len(models)

importances_mean = np.zeros(len(FEATURES))
importances_std  = np.zeros(len(FEATURES))

for model in models:
    results = permutation_importance(
        model,
        X_test_scaled,
        y_test,
        n_repeats=30,
        random_state=42,
        scoring='neg_mean_squared_error'
    )
    importances_mean += results.importances_mean
    importances_std  += results.importances_std

importances_mean /= n_ensembles
importances_std  /= n_ensembles

imp_df = pd.DataFrame({
    'feature': FEATURES,
    'mean_import': importances_mean,
    'std_import': importances_std
}).sort_values('mean_import', ascending=False).reset_index(drop=True)

TOP_N = 20
top20 = imp_df.head(TOP_N).copy()

pretty_names = {
    'avg_magnetic_moment': 'Avg Magnetic Moment',
    'Average_Weight': 'Avg Atomic Weight',
    'Magnetic_proportion': 'Prop. of High Néel Temp Elements',
    'Average_Electronegativity': 'Avg Electronegativity',
    'Avg_Atomic_Number': 'Avg Atomic Number',
    'Entropy': 'Avg Entropy',
    'Rare_Earth_proportion': 'Proportion of Rare Earths',
    'average_group': 'Avg Periodic Group',
    'average_period': 'Avg Periodic Period',
}
top20['label'] = top20['feature'].map(pretty_names).fillna(top20['feature'])

plt.figure(figsize=(9, 6))
plt.barh(
    top20['label'][::-1],
    top20['mean_import'][::-1],
    xerr=top20['std_import'][::-1],
    color='royalblue',
    edgecolor='black',
    capsize=3
)
plt.xlabel('Average Permutation Importance\n(Ensemble Mean ± 1 SD)', fontsize=13)
plt.title('Top-20 Feature Importances — Neural Network Ensemble (Néel Temperature Prediction)', fontsize=15)
plt.grid(axis='both', linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()